In [218]:
from bs4 import BeautifulSoup
from datetime import datetime
from csv import writer
import requests
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [216]:
#Variable for enumerate the hotels in the order of the pages
actual_index = 0
#Variable for the number of iterations
counter = 0
#Variable for change the page 
aumento = 0
#Variable for the limit to iterate - in the first iteration it will change to the number of hotels that tripadvisor has 
limit = 9999999
#Url of tripadvisor
url = 'https://www.tripadvisor.com'
#Url of the hotels of tripadvisor
url_bogota = '/Hotels-g294074-Bogota-Hotels.html'
#Dictionary created to save the name of the hotels and do not repeat any of them
dict_names_hotels = {}
#Writer for the csv file
with open('iceberg_test.csv', 'w', encoding = 'utf-8', newline ='') as f: 
    thewriter = writer(f)
    header = ['Name', 'Rank', 'ID', 'URL Tripadvisor','Avg Price', 'Phone number', 
             'Address', 'Rate Tripadvisor', 'Location Rate', 'Cleanliness Rate', 'Service Rate', 'Value Rate',
             'Property Amenities', 'Room Features', 'Room Types', 'Timestamp']
    thewriter.writerow(header)
    #Cycle for iterate above all the pages for bogota hotels
    while counter < limit:
        #Set the url for the first HTTP Request in order to get the principal page with the list of hotels
        url_total = url+url_bogota
        print(url_total)
        #Set the headers
        headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
        #Make the HTTP Request
        html = requests.get(url_total, headers=headers)
        html_text = html.text
        #Instance the first BeautifulSoup object to search in the listing page
        soup = BeautifulSoup(html_text, 'lxml')
        #Finding all the divs in which the information of the hotels are declared
        div_tags = soup.find_all('div', class_='meta_listing ui_columns large_thumbnail_mobile')
        #Find the total of hotels that Bogota has. This will help us with the iteration
        limit = int(soup.find('span', class_='qrwtg').text.split()[0].replace(',',''))
        for div in div_tags:
            #Array to save the info
            info = []
            #title_text = title.text.split('.')[1].lstrip(' ') if title != None else None
            #Add 1 to the index counter
            actual_index += 1
            #Add 1 to the counter
            counter += 1
            print('Counter', counter)
            #Timestamp
            x = time.time()
            dt = datetime.fromtimestamp(x).strftime("%m/%d/%Y, %H:%M:%S")
            print(dt)
            #Here, we get the Hotel's URL from TripAdvisor, Hotel ID and Hotel rank
            rank = actual_index
            info.append(rank)
            hotel_id = div['data-listingkey']
            info.append(hotel_id)
            url_actual = div['data-url']
            #We call it details due to the information that we will get from that link
            url_details = url + url_actual
            info.append(url_details)
            #Then, we make the HTTP Request again of that URL to get the HTML code
            details = requests.get(url_details, headers=headers)
            details_text = details.text
            #Construct another instance of Beautiful Soup in order to navigate the Hotel detailed page
            soup2 = BeautifulSoup(details_text, 'lxml')
            #Search the title - name of the hotel
            title = soup2.find('h1', class_='QdLfr b d Pn')
            title_text = title.text if title != None else None
            if title_text in dict_names_hotels:
                continue
            else:
                dict_names_hotels[title_text] = True
            info.insert(0, title_text)
            #Search the price of the hotel
            price = soup2.find('span', class_='YiGCY')
            price_final = int(price.text.split()[-1].replace(',','')) if price != None and price.text.split()[-1]!='deals' else None
            info.append(price_final)
            #Search the phone number of the hotel
            phoneNumber = soup2.find('span', class_='zNXea NXOxh NjUDn')
            phoneNumber_text = phoneNumber.text if phoneNumber != None else None
            info.append(phoneNumber_text)
            #Search the address of the hotel
            address = soup2.find('span', class_='fHvkI PTrfg')
            address_text = address.text if address != None else None
            info.append(address_text)
            #Search the general rating of the hotel
            rate = soup2.find('span', class_='uwJeR P')
            rate_text = float(rate.text) if rate != None else None
            info.append(rate_text)
            #Search other ratings like Location, Cleanliness, etc.
            otherRatings = soup2.find_all('div', class_='WdWxQ')
            for rating in otherRatings:
                rate_value_text = float(rating.find_next('span').find_next('span').text)
                info.append(rate_value_text)
            if len(otherRatings) <4 or not otherRatings:
                for i in range(0, 4 - len(otherRatings) if otherRatings else 4, 1):
                    info.append(None)
            #Search differents properties of the hotel
            #The next line is to get the properties titles: Amenities, Room features and Room types
            properties = soup2.find_all('div', class_='aeQAp S5 b Pf ME')
            #Have to minimize the array because not all of the divs returned are of our interest
            final_properties = properties[0:4]
            #Search the properties in a loop. As the titles case, we have to stop at the third iteration because the other div's content
            #won't be saved
            features = soup2.find_all('div', class_='OsCbb K')
            i = 0
            for feature in features:
                actual = ''
                real_features = feature.find_all_next('div', class_='yplav f ME H3 _c')
                indexx = 0
                lenFeatures = len(real_features)
                for j in real_features:
                    #Index to know the last feature
                    actual += j.text +', ' if indexx < lenFeatures - 2 else j.text
                info.append(actual)
                i+=1
                if i == 3:
                    break
            if len(features) <3 or not features:
                for i in range(0, 3 - len(features) if features else 3, 1):
                    info.append(None)
            #Append the timestamp
            info.append(dt)
            #Write the row of the hotel
            print('---------------------------------------------------------------------------------')
            print(info)
            print('---------------------------------------------------------------------------------')
            thewriter.writerow(info)
        #Set the incremental variable for search hotels in the other page. In tripadvisor case, each page is +30 from the previous 
        #one in the URL. Also, the sintaxis of the URL changes as the following string
        aumento += 30
        url_bogota = '/Hotels-g294074'+'-oa'+str(aumento)+'-Bogota-Hotels.html'
        print('AAAA', url_bogota)
        print(actual_index)
        print(limit)
        #new counter to equalize to limit

https://www.tripadvisor.com/Hotels-g294074-Bogota-Hotels.html
Counter 1
12/26/2022, 18:30:30
---------------------------------------------------------------------------------
['Grand Hyatt Bogota', 1, 'a435c11c-b279-40b5-848f-b5ae535d904c', 'https://www.tripadvisor.com/Hotel_Review-g294074-d13288526-Reviews-Grand_Hyatt_Bogota-Bogota.html', 754440, '(601) 6541234', 'Calle 24 A, 57-60, Bogota 111321 Colombia', 4.5, 4.6, 4.9, 4.7, 4.6, "Free parking, Free High Speed Internet (WiFi), Pool, Fitness Center with Gym / Workout Room, Bar / lounge, Children's television networks, Highchairs available, Pets Allowed ( Dog / Pet Friendly ), Valet parking, Wifi, Hot tub, Pool / beach towels, Indoor pool, Heated pool, Yoga classes, Fitness / spa locker rooms, Fitness classes, Personal trainer, Sauna, Coffee shop, Restaurant, Breakfast available, Breakfast buffet, Complimentary welcome drink, Kids' meals, Wine / champagne, Airport transportation, Car hire, Taxi service, Business Center with Internet A

---------------------------------------------------------------------------------
['Hotel Vilar America', 4, '4b023d1a-880d-4a25-9402-f49c60c7d55a', 'https://www.tripadvisor.com/Hotel_Review-g294074-d659553-Reviews-Hotel_Vilar_America-Bogota.html', 107547, None, 'Calle 66 8 - 23 Barrio Chapinero, Bogota 110231 Colombia', 4.5, 4.4, 4.6, 4.5, 4.4, 'Free High Speed Internet (WiFi), Free breakfast, Bicycle rental, Children Activities (Kid / Family Friendly), Highchairs available, Airport transportation, Conference facilities, Banquet room, Wifi, Restaurant, Breakfast available, Breakfast buffet, Breakfast in the room, Complimentary Instant Coffee, Happy hour, Meeting rooms, Couples massage, Foot massage, Full body massage, Hand massage, Head massage, Manicure, Massage, Massage chair, Neck massage, 24-hour security, Baggage storage, Concierge, Currency exchange, Gift shop, Newspaper, Non-smoking hotel, 24-hour front desk, Dry cleaning, Laundry service, Ironing service, Shoeshine, Blackout c

---------------------------------------------------------------------------------
['Hotel Movich Buro 26', 10, 'd3849f28-6edb-4f0e-8ec9-18fd8d9788bb', 'https://www.tripadvisor.com/Hotel_Review-g294074-d6514692-Reviews-Hotel_Movich_Buro_26-Bogota.html', 199528, '(601) 4824466', 'Av. El Dorado N° 102 - 20, Bogota Colombia', 4.0, 4.5, 4.7, 4.2, 4.3, 'Free parking, Free High Speed Internet (WiFi), Fitness Center with Gym / Workout Room, Free breakfast, Free airport transportation, Business Center with Internet Access, Conference facilities, Spa, Parking, Wifi, Fitness / spa locker rooms, Sauna, Bar / lounge, Restaurant, Breakfast available, Breakfast buffet, Free shuttle or taxi services, Airport transportation, Shuttle bus service, Banquet room, Meeting rooms, Massage, Baggage storage, Concierge, Non-smoking hotel, 24-hour front desk, Dry cleaning, Laundry service, Ironing service, Air conditioning, Room service, Safe, Wake-up service / alarm clock, Minibar, Flatscreen TV, Non-smoking roo

---------------------------------------------------------------------------------
['Bogota Plaza Hotel', 14, 'c99a3b5a-a96a-42b9-b222-f034628f7265', 'https://www.tripadvisor.com/Hotel_Review-g294074-d304496-Reviews-Bogota_Plaza_Hotel-Bogota.html', 236312, None, 'Carrera 18a 100 41 Carrera 18A # 100-41, Bogota 110111 Colombia', 4.5, 4.4, 4.5, 4.5, 4.2, "Parking, Free High Speed Internet (WiFi), Hot tub, Fitness Center with Gym / Workout Room, Free breakfast, Children's television networks, Highchairs available, Airport transportation, Wifi, Fitness / spa locker rooms, Sauna, Bar / lounge, Restaurant, Breakfast available, Breakfast buffet, Breakfast in the room, Taxi service, Business Center with Internet Access, Conference facilities, Banquet room, Meeting rooms, Photo copier / fax In business center, Spa, Couples massage, Facial treatments, Full body massage, Massage, Steam room, 24-hour security, Baggage storage, Concierge, Executive lounge access, Non-smoking hotel, Umbrella, 24-hour

---------------------------------------------------------------------------------
['NH Bogota Urban 26 Royal', 19, '3bcb05af-2f74-41e4-8ba8-953a2036449e', 'https://www.tripadvisor.com/Hotel_Review-g294074-d5308199-Reviews-NH_Bogota_Urban_26_Royal-Bogota.html', 166036, '+1 212-245-5462', 'Carrera 33 No. 25F-18, Bogota 111321 Colombia', 4.5, 4.3, 4.5, 4.5, 4.4, 'Free High Speed Internet (WiFi), Fitness Center with Gym / Workout Room, Free breakfast, Bicycle rental, Babysitting, Children Activities (Kid / Family Friendly), Airport transportation, Business Center with Internet Access, Wifi, Bar / lounge, Restaurant, Breakfast available, Breakfast buffet, Breakfast in the room, Conference facilities, Banquet room, Meeting rooms, Baggage storage, Concierge, Non-smoking hotel, 24-hour front desk, Dry cleaning, Laundry service, Fireplace, Housekeeping, Room service, Safe, Microwave, Minibar, Flatscreen TV, Bath / shower, Telephone, Laptop safe, Refrigerator, Complimentary toiletries, Hair drye

...

---------------------------------------------------------------------------------
['Casa Hotel Bogota Ferial', 1383, 'dbdcf833-9f8d-4038-a863-f43c3d6236b9', 'https://www.tripadvisor.com/Hotel_Review-g294074-d11704307-Reviews-Casa_Hotel_Bogota_Ferial-Bogota.html', None, None, '5219 Calle 22 Bis Calle 22bis # 44a-19, Bogota 111321 Colombia', None, None, None, None, None, None, None, None, '12/26/2022, 19:01:37']
---------------------------------------------------------------------------------
Counter 1384
12/26/2022, 19:01:38
---------------------------------------------------------------------------------
['Balmoral 103', 1384, '2fa634e9-f433-4821-9296-b9502cbf0572', 'https://www.tripadvisor.com/Hotel_Review-g294074-d23353964-Reviews-Balmoral_103-Bogota.html', None, None, 'Carrera 18 A 103 -29, Bogota 110111 Colombia', None, None, None, None, None, 'Free parking, Free High Speed Internet (WiFi), Internet, Pets Allowed ( Dog / Pet Friendly ), Concierge, ', None, None, '12/26/2022, 19:01:

---------------------------------------------------------------------------------
['Hostel By Parque 93 & Virrey', 1401, 'bceff648-1535-4171-b249-0b23b3995991', 'https://www.tripadvisor.com/Hotel_Review-g294074-d21082573-Reviews-Hostel_By_Parque_93_Virrey-Bogota.html', None, None, 'Calle 93 19 05, Bogota 110221 Colombia', None, None, None, None, None, 'Paid private parking on-site, Parking, Free High Speed Internet (WiFi), 24-hour security, Non-smoking hotel, Dining area, Telephone, Microwave, Refrigerator, Stovetop, Kitchenware, Bath / shower, City view, Non-smoking rooms, ', 'Dining area, Telephone, Microwave, Refrigerator, Stovetop, Kitchenware, Bath / shower, City view, Non-smoking rooms, ', 'City viewNon-smoking rooms', '12/26/2022, 19:01:58']
---------------------------------------------------------------------------------
Counter 1402
12/26/2022, 19:01:59
---------------------------------------------------------------------------------
['Hostal El Tocororo', 1402, '3d00aeb5-a2fa

---------------------------------------------------------------------------------
['Morph Chico', 1415, '722edb96-216b-4842-911b-78603251dd97', 'https://www.tripadvisor.com/Hotel_Review-g294074-d24072793-Reviews-Morph_Chico-Bogota.html', None, None, None, None, None, None, None, None, None, None, None, '12/26/2022, 19:02:17']
---------------------------------------------------------------------------------
Counter 1416
12/26/2022, 19:02:17
---------------------------------------------------------------------------------
['Casa de Sion', 1416, '96fbc76d-2ddd-47cd-907c-7a043c1634c6', 'https://www.tripadvisor.com/Hotel_Review-g294074-d19778044-Reviews-Casa_de_Sion-Bogota.html', None, None, 'Carrera 14b #111-19, Bogota 110111 Colombia', None, None, None, None, None, None, None, None, '12/26/2022, 19:02:17']
---------------------------------------------------------------------------------
Counter 1417
12/26/2022, 19:02:19
---------------------------------------------------------------------

---------------------------------------------------------------------------------
['Norte de Bogota Tranquilo', 1430, '143a98fc-696a-412f-b3ef-e03691c0161e', 'https://www.tripadvisor.com/Hotel_Review-g294074-d23397729-Reviews-Norte_de_Bogota_Tranquilo-Bogota.html', None, None, 'Calle 168a 58a 87, Bogota 111156 Colombia', None, None, None, None, None, None, None, None, '12/26/2022, 19:02:32']
---------------------------------------------------------------------------------
Counter 1431
12/26/2022, 19:02:33
---------------------------------------------------------------------------------
['Like Home', 1431, '8600f4b2-70a8-49ce-b5a0-219821f9d926', 'https://www.tripadvisor.com/Hotel_Review-g294074-d13476802-Reviews-Like_Home-Bogota.html', None, None, '466 Calle 61, Bogota 110231 Colombia', None, None, None, None, None, 'Secured parking, Free High Speed Internet (WiFi), Non-smoking hotel, 24-hour front desk, Telephone, Clothes rack, Iron, Coffee / tea maker, Microwave, Refrigerator, Bath / 

---------------------------------------------------------------------------------
['Hotel Ayenda Bioma 1097', 1445, 'd8789823-a955-47a0-9d10-05e0faae209a', 'https://www.tripadvisor.com/Hotel_Review-g294074-d23453997-Reviews-Hotel_Ayenda_Bioma_1097-Bogota.html', 84024, None, None, None, None, None, None, None, None, None, None, '12/26/2022, 19:02:50']
---------------------------------------------------------------------------------
Counter 1446
12/26/2022, 19:02:51
---------------------------------------------------------------------------------
['Mi Casita', 1446, 'c9ca07d9-f13f-4868-8504-cbc9c56efee5', 'https://www.tripadvisor.com/Hotel_Review-g294074-d15584586-Reviews-Mi_Casita-Bogota.html', None, None, 'Cra. 80D #13-67, El Tintal, Bogota 110821 Colombia', None, None, None, None, None, None, None, None, '12/26/2022, 19:02:51']
---------------------------------------------------------------------------------
AAAA /Hotels-g294074-oa1470-Bogota-Hotels.html
1446
1446


# Pre procesamiento de datos numéricos

In this part of the notebook we will change the None values of all the rates (Tripadvisor, Location, Cleanliness, Service and Value) for the median of the respective rate and export the result dataframe into a csv file.

In [285]:
# Se cargan los datos. 
df_original = pd.read_csv('iceberg_test.csv', sep=',', encoding = 'utf-8', index_col = 0, na_values=['None'])
df_hotels = df_original.copy()
df_hotels.head(5)

,Rank,ID,URL Tripadvisor,Avg Price,Phone number,Address,Rate Tripadvisor,Location Rate,Cleanliness Rate,Service Rate,Value Rate,Property Amenities,Room Features,Room Types,Timestamp
Name,,,,,,,,,,,,,,,
Grand Hyatt Bogota,1,a435c11c-b279-40b5-848f-b5ae535d904c,https://www.tripadvisor.com/Hotel_Review-g2940...,754440.0,(601) 6541234,"Calle 24 A, 57-60, Bogota 111321 Colombia",4.5,4.6,4.9,4.7,4.6,"Free parking, Free High Speed Internet (WiFi),...","Allergy-free room, Blackout curtains, Air cond...","Mountain view, City view, Bridal suite, Non-sm...","12/26/2022, 18:30:30"
Hilton Garden Inn Bogota Airport,2,b470ba44-b834-4ac1-b2b8-fbdb7ae2325d,https://www.tripadvisor.com/Hotel_Review-g2940...,305849.0,+1 855-618-4697,"Carrera 82 No. 25G 84, Bogota 110931 Colombia",4.5,4.8,4.9,4.6,4.7,"Free parking, Free High Speed Internet (WiFi),...","Allergy-free room, Soundproof rooms, Air condi...","Mountain view, City view, Non-smoking rooms, S...","12/26/2022, 18:30:32"
93 Luxury Suites & Residences,3,9e17675e-bf07-48c1-ac7f-203e6a89790e,https://www.tripadvisor.com/Hotel_Review-g2940...,502822.0,+5717459093,"Carrera 13A 93-51, Bogota 110221 Colombia",5.0,4.9,5.0,4.8,4.7,"Free parking, Free High Speed Internet (WiFi),...","Blackout curtains, Soundproof rooms, Additiona...","Mountain view, City view, Landmark view, Non-s...","12/26/2022, 18:30:33"
Hotel Vilar America,4,4b023d1a-880d-4a25-9402-f49c60c7d55a,https://www.tripadvisor.com/Hotel_Review-g2940...,107547.0,NaN,"Calle 66 8 - 23 Barrio Chapinero, Bogota 11023...",4.5,4.4,4.6,4.5,4.4,"Free High Speed Internet (WiFi), Free breakfas...","Blackout curtains, Desk, Housekeeping, Interco...","Bridal suite, Non-smoking rooms, Suites, Famil...","12/26/2022, 18:30:35"
NH Collection Bogota Hacienda Royal,5,97984fa1-c031-4af3-92a4-3ce4577203f4,https://www.tripadvisor.com/Hotel_Review-g2940...,213950.0,+1 212-245-5462,"Calle 114 No 6 - 02, Bogota Colombia",4.5,4.7,4.7,4.8,4.4,"Free parking, Free High Speed Internet (WiFi),...","Soundproof rooms, Air conditioning, Room servi...","Bridal suite, Non-smoking rooms, Suites, Famil...","12/26/2022, 18:30:36"


In [286]:
print(df_hotels.mean(numeric_only = True))

Rank                   722.468610
Avg Price           188253.157248
Rate Tripadvisor         3.833123
Location Rate            4.233521
Cleanliness Rate         4.148063
Service Rate             4.092320
Value Rate               4.000000
dtype: float64


In [287]:
print(df_hotels.mean(numeric_only = True)['Rank'])

722.4686098654709


In [288]:
print(df_hotels['Avg Price'])
print(df_hotels['Rate Tripadvisor'])
print(df_hotels['Location Rate'])
print(df_hotels['Cleanliness Rate'])
print(df_hotels['Service Rate'])
print(df_hotels['Value Rate'])

Name
Grand Hyatt Bogota                     754440.0
Hilton Garden Inn Bogota Airport       305849.0
93 Luxury Suites & Residences          502822.0
Hotel Vilar America                    107547.0
NH Collection Bogota Hacienda Royal    213950.0
                                         ...   
Habitaciones                                NaN
Dona Luz                                    NaN
Aloha Calle 80                              NaN
Hotel Ayenda Bioma 1097                 84024.0
Mi Casita                                   NaN
Name: Avg Price, Length: 1338, dtype: float64
Name
Grand Hyatt Bogota                     4.5
Hilton Garden Inn Bogota Airport       4.5
93 Luxury Suites & Residences          5.0
Hotel Vilar America                    4.5
NH Collection Bogota Hacienda Royal    4.5
                                      ... 
Habitaciones                           NaN
Dona Luz                               NaN
Aloha Calle 80                         NaN
Hotel Ayenda Bioma 1097      

In [289]:
df_hotels['Avg Price'] = df_hotels['Avg Price'].fillna(df_hotels.median(numeric_only = True)['Avg Price'])
df_hotels['Rate Tripadvisor'] = df_hotels['Rate Tripadvisor'].fillna(df_hotels.median(numeric_only = True)['Rate Tripadvisor'])
df_hotels['Location Rate'] = df_hotels['Location Rate'].fillna(df_hotels.median(numeric_only = True)['Location Rate'])
df_hotels['Cleanliness Rate'] = df_hotels['Cleanliness Rate'].fillna(df_hotels.median(numeric_only = True)['Cleanliness Rate'])
df_hotels['Service Rate'] = df_hotels['Service Rate'].fillna(df_hotels.median(numeric_only = True)['Service Rate'])
df_hotels['Value Rate'] = df_hotels['Value Rate'].fillna(df_hotels.median(numeric_only = True)['Value Rate'])

In [290]:
print(df_hotels['Avg Price'])
print(df_hotels['Rate Tripadvisor'])
print(df_hotels['Location Rate'])
print(df_hotels['Cleanliness Rate'])
print(df_hotels['Service Rate'])
print(df_hotels['Value Rate'])

Name
Grand Hyatt Bogota                     754440.0
Hilton Garden Inn Bogota Airport       305849.0
93 Luxury Suites & Residences          502822.0
Hotel Vilar America                    107547.0
NH Collection Bogota Hacienda Royal    213950.0
                                         ...   
Habitaciones                           145708.0
Dona Luz                               145708.0
Aloha Calle 80                         145708.0
Hotel Ayenda Bioma 1097                 84024.0
Mi Casita                              145708.0
Name: Avg Price, Length: 1338, dtype: float64
Name
Grand Hyatt Bogota                     4.5
Hilton Garden Inn Bogota Airport       4.5
93 Luxury Suites & Residences          5.0
Hotel Vilar America                    4.5
NH Collection Bogota Hacienda Royal    4.5
                                      ... 
Habitaciones                           4.0
Dona Luz                               4.0
Aloha Calle 80                         4.0
Hotel Ayenda Bioma 1097      

In [292]:
df_hotels.to_csv('hotels_fixed.csv', encoding='utf-8')